# FlowMatchNet with OxidizedVision

This notebook demonstrates the end-to-end process of converting a FlowMatchNet model using OxidizedVision, and then running it in Rust.

## 1. Setup

First, let's install the necessary libraries.

In [ ]:
!pip install torch torchvision torchaudio
!pip install -e ../python_client

## 2. Define and Train a (dummy) FlowMatchNet model

For this example, we'll use a simplified version of a FlowMatch-like model. In a real-world scenario, you would have your own pre-trained model.

In [ ]:
import torch
import torch.nn as nn
import os

class FlowMatchNet(nn.Module):
    def __init__(self):
        super(FlowMatchNet, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1),
        )
        self.flow_head = nn.Conv2d(32, 2, 1) # Output 2 channels for flow (x, y)

    def forward(self, x):
        features = self.feature_extractor(x)
        return self.flow_head(features)

# Save the model definition to a file
model_code = '''
import torch
import torch.nn as nn

class FlowMatchNet(nn.Module):
    def __init__(self):
        super(FlowMatchNet, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1),
        )
        self.flow_head = nn.Conv2d(32, 2, 1)

    def forward(self, x):
        features = self.feature_extractor(x)
        return self.flow_head(features)
'''
os.makedirs('flowmatch_model', exist_ok=True)
with open('flowmatch_model/model.py', 'w') as f:
    f.write(model_code)

# Instantiate and save a checkpoint
model = FlowMatchNet()
os.makedirs('checkpoints', exist_ok=True)
torch.save(model.state_dict(), 'checkpoints/flowmatch_ckpt.pt')

print('FlowMatchNet model and checkpoint saved.')

## 3. Create a Configuration File

Now, we'll create a `config.yml` file to tell OxidizedVision how to convert our model.

In [ ]:
import yaml

config = {
    'model': {
        'path': 'flowmatch_model/model.py',
        'class_name': 'FlowMatchNet',
        'input_shape': [1, 3, 128, 128],
        'checkpoint': 'checkpoints/flowmatch_ckpt.pt'
    },
    'export': {
        'opset_version': 14,
        'do_constant_folding': True
    }
}

with open('flowmatch_config.yml', 'w') as f:
    yaml.dump(config, f)

print('Configuration file 'flowmatch_config.yml' created.')

## 4. Run the Conversion

With the model and config ready, we can now use the `oxidizedvision` CLI to perform the conversion.

In [ ]:
!oxidizedvision convert --config flowmatch_config.yml

## 5. Package the Model into a Rust Crate

Next, we'll package the converted ONNX model into a self-contained Rust crate using the `tract` runtime.

In [ ]:
!oxidizedvision package --onnx out/model.onnx --runner tract --out ../rust_runtime/packaged/flowmatch_tract

## 6. Build and Run in Rust

Finally, we can navigate to the newly created Rust crate, build it, and run it.

You can do this from your terminal:

In [ ]:
print('To build and run, execute the following in your terminal:')
print('cd ../rust_runtime/packaged/flowmatch_tract')
print('cargo build --release')
# This is just an example name, you would create a binary to run.
print('./target/release/your_binary_name')